# VacationPy
----

In this notebook the recently created csv file with weather information of  56?? cities around the globe is going to be used. The main idea is to finf the best places to have an idillyc vacation time. But before getting info of such kinf of vacations lets prepare everything required for this analysis.

In [25]:
#Lets import modules (Dependencies)
import pandas as pd
import requests
import json
import gmaps
import numpy as np
import matplotlib as plt

#Lets import the api key
from config import g_key

#Lets configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
#Lets call the df with weather info
cities_path = "output_data/cities_weather.csv"

In [3]:
cities_df = pd.read_csv(cities_path)
cities_df.head()

,City,Latitude,Longitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind Speed
0,goma,-1.68,29.22,65.79,65.79,65,56,2.21
1,constitucion,-35.33,-72.42,52.63,52.63,75,69,5.79
2,shenzhen,22.55,114.07,83.26,84.00,98,4,3.49
3,jamestown,42.10,-79.24,66.40,66.99,52,1,16.11
4,homer,59.64,-151.55,53.53,55.40,71,75,5.82


In [5]:
#Lets see the size
cities_df.shape

(592, 8)

In [6]:
#Check coordinates
print(f"The maximum latitude is {cities_df['Latitude'].max()} and the minimum is {cities_df['Latitude'].min()}")
print(f"The maximum longitude is {cities_df['Longitude'].max()} and the minimum is {cities_df['Longitude'].min()}")

The maximum latitude is 78.22 and the minimum is -54.8
The maximum longitude is 179.32 and the minimum is -179.17


## Heat map of world's humidity(04-09-2020)
- - - -

To generate a heatmap, whose colors are going to be defined bu the humidity of each of the cities in the dataframe, the gmaps API. Gmaps is a plugin for Jupyter for embedding Google Maps in a notebook, and is designed as a data visualization tool.

In order to generate a mao is necessary to use the function gmaps.figure, and the it is necessary to add a layer that contains a function named gmaps.heatmap_layer. Is in this last fucntion where the geolocation coordinates and the variable that is going to define intensity in each geolocation of the heat map. 

Lets create the humidity heat map.

In [7]:
#Lets check what kind of elemets are in cities_df
cities_df.dtypes

City                object
Latitude           float64
Longitude          float64
Temperature        float64
Max Temperature    float64
Humidity             int64
Cloudiness           int64
Wind Speed         float64
dtype: object

In [8]:
#Since all the variables are in the correct format no transformation has tobe done
#Lets extract the coordinates
# Store latitude and longitude in locations
locations = cities_df[["Latitude", "Longitude"]]

# Store humidity in humidity
humidity = cities_df["Humidity"]

# Keep maximum humidity
max_hum = cities_df["Humidity"].max()

In [9]:
#Lets create the map
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity= max_hum, point_radius=3.0)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

## Selecting the best cities for an idillyc vacation
- - - -

There is a concept known as Thermal comfort, and is defined as the condition of mind that expresses satisfaction with the thermal environment and is assessed by subjective evaluation (wikipedia, ANSI/ASHRAE Standard 55). Accoridng to to Covarrubias (2012), the temperature confort range is between 21.1°C and 27.5°, but other authors propose a range between 18°c abd 20°. Also, Covarrubias (2012) metions an international standard and an ISO norm that states that in order to keep air quality inside a building humidity must be between  dty that make  that is between 30% and 70%. With respect to wind speed, the web page (https://www.seiscubos.com/conocimiento/confort-termico-y-cuerpo-humano) at templated climates the best wind speed is at is less than 0.25m/s. Therefore, in this section cities with weather parameters in hte explianed above are going to be selected. In the next table, the ranges as specified in the Imperial Units. Also only cities without a low amount of clouds are going to be selected

| Variable | minimun | maximum |
| - - - - -|- - - - -| - - - - |
|Temperature | 69.98 F| 81.5  F |
|Humidity  | 30 %    | 70%     |
|Wind Speed| 0.56 mph| 10 mph  |
|Cloudiness| 0 %     | 10 %    |




In [10]:
#Lets filter cities_df by maximum values
vacation_df = cities_df[((cities_df["Max Temperature"] <= 81.50) & (cities_df["Humidity"] <= 70) & (cities_df["Wind Speed"] <= 10) & (cities_df["Cloudiness"] <= 10))]
vacation_df.shape
                  

(57, 8)

In [11]:
#Lets filter vacatios df by minimum values
vacation_df = vacation_df[((vacation_df["Max Temperature"] >= 69.98) & (vacation_df["Humidity"] >= 30) & (vacation_df["Wind Speed"] >= 0.56))]
vacation_df.shape

(17, 8)

In [12]:
vacation_df

,City,Latitude,Longitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind Speed
43,albany,42.60,-73.97,72.52,75.99,50,5,6.71
65,bethel,41.37,-73.41,78.64,80.60,36,1,8.05
71,broome,42.25,-75.83,71.28,73.99,40,1,9.17
128,red wing,44.56,-92.53,73.80,73.99,33,1,4.70
176,saurimo,-9.66,20.39,71.78,71.78,36,5,2.17
182,arraial do cabo,-22.97,-42.03,77.00,77.00,69,0,6.93
267,bud,39.45,-86.18,75.06,75.20,33,1,5.82
276,nurota,40.56,65.69,71.60,71.60,30,0,9.17
303,eureka,40.80,-124.16,70.68,73.99,64,1,6.93
443,marzuq,14.40,46.47,79.25,79.25,41,5,3.44


# Adding the Country

As it cas be observed only 17 cities fullfill the human termal confort parameters, and therefore they can be considered as cities for a nice vacation time. Also, an inspection of the dataframe shows that the country is missing for all the cities. Then, a geocode querry is going to be done using only the name of the city in each row. To do so, an iteration over the data frame rows id going to be done. Also the coordinates are goingo to be retreived again

In [ ]:
#First we define new three new columns
vacation_df['Country']= " "
vacation_df["nlat"]= " "
vacation_df["nlng"]= " "

In [13]:
#Dictionary with query parameters for geocoding search
params = {"key": g_key}


# Iterate through rows to recover the name of the city in each row
for index, row in vacation_df.iterrows():
    # get lat, lng from df
    target_city = row['City']
    # change location each iteration while leaving original params in place
    params["address"] = target_city

    # Use the search term inside target_city
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    # make request
    response = requests.get(base_url, params = params)
    
    #Lets extract info
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        vacation_df.loc[index, "Country"] = response.json()['results'][0]['address_components'][3]['short_name']
        vacation_df.loc[index, "nlat"] = response.json()['results'][0]['geometry']['location']['lat']
        vacation_df.loc[index, "nlng"] = response.json()['results'][0]['geometry']['location']['lng']
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [14]:
vacation_df

,City,Latitude,Longitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,nlat,nlng
43,albany,42.60,-73.97,72.52,75.99,50,5,6.71,US,42.652579,-73.756232
65,bethel,41.37,-73.41,78.64,80.60,36,1,8.05,Tulsa County,36.059996,-95.797855
71,broome,42.25,-75.83,71.28,73.99,40,1,9.17,Dickinson,42.134880,-75.908855
128,red wing,44.56,-92.53,73.80,73.99,33,1,4.70,Owasso,36.291764,-95.848629
176,saurimo,-9.66,20.39,71.78,71.78,36,5,2.17,AO,-9.661217,20.390460
182,arraial do cabo,-22.97,-42.03,77.00,77.00,69,0,6.93,BR,-22.967337,-42.026810
267,bud,39.45,-86.18,75.06,75.20,33,1,5.82,Labette County,37.337141,-95.278260
276,nurota,40.56,65.69,71.60,71.60,30,0,9.17,UZ,40.563118,65.701514
303,eureka,40.80,-124.16,70.68,73.99,64,1,6.93,KS,37.823917,-96.289170
443,marzuq,14.40,46.47,79.25,79.25,41,5,3.44,Leon County,30.464161,-84.282242


As it can be observed there is a correction done to all the cities coordinates. However, some of them are important, such as Bethel, Bud,Marzuq, etc. Therefore, the old coordinates are going to be used.


## Searching for a hotel in the best cities for vacation
- - - - 

Now, it is going to be searched if in the 17 cities selected is an hotel. To do so the Google Places API is going to be used. The selested kind of query is a Nearby Search requests.  A Nearby Search allows to search for places within a specified area. It is possible to refine the search request by supplying keywords or specifying the type of place serached for.

A Nearby Search request is an HTTP URL of the following form:

https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters

In our case the ouput is a json file. Also, the required parameters are the API key, a keyword that os a term to be matched against all content that Google has indexed for a place. A name which is a a term to be matched against all content that Google has indexed for a place and is equivalent  to keyword. opennow, returns only those places that are open for business at the time the query is sent, rankby specifies the order in which results are listed, etc.

In this case it is going to be used the following parameters: "radius", "types", and the"key".

In the next cells the procedure is shown.


In [15]:
##Lets start the query for the vactions_df
#To do so, first we need to add new columns to vactions_df, where info is going to be stores
# Add columns for lat, lng, airport name, airport address, airport rating
# Note that we used "" to specify initial entry.
vacation_df["Hotel Name"] = ""
vacation_df["Hotel Address"] = ""
vacation_df.head()

,City,Latitude,Longitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,nlat,nlng,Hotel Name,Hotel Address
43,albany,42.60,-73.97,72.52,75.99,50,5,6.71,US,42.652579,-73.756232,,
65,bethel,41.37,-73.41,78.64,80.60,36,1,8.05,Tulsa County,36.059996,-95.797855,,
71,broome,42.25,-75.83,71.28,73.99,40,1,9.17,Dickinson,42.134880,-75.908855,,
128,red wing,44.56,-92.53,73.80,73.99,33,1,4.70,Owasso,36.291764,-95.848629,,
176,saurimo,-9.66,20.39,71.78,71.78,36,5,2.17,AO,-9.661217,20.390460,,


In [16]:
# Lets define the query

#parameters dictionary to update each iteration the keyword lodgind is added to ensure it is a hotel and not another kind of 
#bussines that might have in its name the world hotel
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to search for hotels
for index, row in vacation_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request
    name_address = requests.get(base_url, params=params)
    

    # convert to json
    name_address = name_address.json()
    

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        vacation_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        vacation_df.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [17]:
#Lets se the result
vacation_df

,City,Latitude,Longitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,nlat,nlng,Hotel Name,Hotel Address
43,albany,42.60,-73.97,72.52,75.99,50,5,6.71,US,42.652579,-73.756232,,
65,bethel,41.37,-73.41,78.64,80.60,36,1,8.05,Tulsa County,36.059996,-95.797855,Best Western Danbury/Bethel,"11 Stony Hill Rd, Bethel"
71,broome,42.25,-75.83,71.28,73.99,40,1,9.17,Dickinson,42.134880,-75.908855,Chenango Valley State Park,"153 State Park Rd, Chenango Forks"
128,red wing,44.56,-92.53,73.80,73.99,33,1,4.70,Owasso,36.291764,-95.848629,St. James Hotel,"406 Main St, Red Wing"
176,saurimo,-9.66,20.39,71.78,71.78,36,5,2.17,AO,-9.661217,20.390460,Resort Uhenha,"Rua da Piscina, bairro Agostinho Neto"
182,arraial do cabo,-22.97,-42.03,77.00,77.00,69,0,6.93,BR,-22.967337,-42.026810,HOSTEL A COROA,"R. Marcílio Dias, 217 - Macedônia, Arraial do ..."
267,bud,39.45,-86.18,75.06,75.20,33,1,5.82,Labette County,37.337141,-95.278260,,
276,nurota,40.56,65.69,71.60,71.60,30,0,9.17,UZ,40.563118,65.701514,Guest House Ruslan Nurata & Tours,"St. Okhunboboyev, h. 2 Nurata UZ"
303,eureka,40.80,-124.16,70.68,73.99,64,1,6.93,KS,37.823917,-96.289170,Best Western Plus Humboldt Bay Inn,"232 W 5th St, Eureka"
443,marzuq,14.40,46.47,79.25,79.25,41,5,3.44,Leon County,30.464161,-84.282242,,


## Pointing the location of an hotel in the selected cities
- - - - 

In this last part, a layer to the humidity map is added. The layer is goingo to contain markers with the location of some of the cities where aj hotel is found from the last request. To do a list of a messages that are going to be shown in each marker is created. The result is shown in the next figure.

In [26]:
#Lets convert the missing values into NAN objects 
vacation_df.replace("", np.nan, inplace=True) 
vacation_df

,City,Latitude,Longitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,nlat,nlng,Hotel Name,Hotel Address
43,albany,42.60,-73.97,72.52,75.99,50,5,6.71,US,42.652579,-73.756232,NaN,NaN
65,bethel,41.37,-73.41,78.64,80.60,36,1,8.05,Tulsa County,36.059996,-95.797855,Best Western Danbury/Bethel,"11 Stony Hill Rd, Bethel"
71,broome,42.25,-75.83,71.28,73.99,40,1,9.17,Dickinson,42.134880,-75.908855,Chenango Valley State Park,"153 State Park Rd, Chenango Forks"
128,red wing,44.56,-92.53,73.80,73.99,33,1,4.70,Owasso,36.291764,-95.848629,St. James Hotel,"406 Main St, Red Wing"
176,saurimo,-9.66,20.39,71.78,71.78,36,5,2.17,AO,-9.661217,20.390460,Resort Uhenha,"Rua da Piscina, bairro Agostinho Neto"
182,arraial do cabo,-22.97,-42.03,77.00,77.00,69,0,6.93,BR,-22.967337,-42.026810,HOSTEL A COROA,"R. Marcílio Dias, 217 - Macedônia, Arraial do ..."
267,bud,39.45,-86.18,75.06,75.20,33,1,5.82,Labette County,37.337141,-95.278260,NaN,NaN
276,nurota,40.56,65.69,71.60,71.60,30,0,9.17,UZ,40.563118,65.701514,Guest House Ruslan Nurata & Tours,"St. Okhunboboyev, h. 2 Nurata UZ"
303,eureka,40.80,-124.16,70.68,73.99,64,1,6.93,KS,37.823917,-96.289170,Best Western Plus Humboldt Bay Inn,"232 W 5th St, Eureka"
443,marzuq,14.40,46.47,79.25,79.25,41,5,3.44,Leon County,30.464161,-84.282242,NaN,NaN


In [27]:
#Lets drop nas from de vacation_df
vacation_df.dropna(subset=['Hotel Name'], inplace=True)
vacation_df 

,City,Latitude,Longitude,Temperature,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,nlat,nlng,Hotel Name,Hotel Address
65,bethel,41.37,-73.41,78.64,80.60,36,1,8.05,Tulsa County,36.059996,-95.797855,Best Western Danbury/Bethel,"11 Stony Hill Rd, Bethel"
71,broome,42.25,-75.83,71.28,73.99,40,1,9.17,Dickinson,42.134880,-75.908855,Chenango Valley State Park,"153 State Park Rd, Chenango Forks"
128,red wing,44.56,-92.53,73.80,73.99,33,1,4.70,Owasso,36.291764,-95.848629,St. James Hotel,"406 Main St, Red Wing"
176,saurimo,-9.66,20.39,71.78,71.78,36,5,2.17,AO,-9.661217,20.390460,Resort Uhenha,"Rua da Piscina, bairro Agostinho Neto"
182,arraial do cabo,-22.97,-42.03,77.00,77.00,69,0,6.93,BR,-22.967337,-42.026810,HOSTEL A COROA,"R. Marcílio Dias, 217 - Macedônia, Arraial do ..."
276,nurota,40.56,65.69,71.60,71.60,30,0,9.17,UZ,40.563118,65.701514,Guest House Ruslan Nurata & Tours,"St. Okhunboboyev, h. 2 Nurata UZ"
303,eureka,40.80,-124.16,70.68,73.99,64,1,6.93,KS,37.823917,-96.289170,Best Western Plus Humboldt Bay Inn,"232 W 5th St, Eureka"
468,korla,41.76,86.15,72.00,72.00,44,0,3.00,CN,41.725891,86.174633,Kangcheng Jian Guo International Hotel,"No 618 East Jiaotong Road Korla Xinjiang, Korla"
505,carbondale,37.73,-89.22,79.14,80.60,36,1,6.93,IL,37.727273,-89.216750,Pine Manor,"26 Pine Lake Dr, Carbondale"
516,kununurra,-15.77,128.73,73.40,73.40,60,0,4.70,AU,-15.604949,128.765468,Freshwater East Kimberley Apartments | Kununur...,"19 Victoria Hwy, Kununurra"


In [28]:
#Create a list with message to display
messages =[]

#Iterate by rows
for index, row in vacation_df.iterrows():
    # get info
    hotel = row["Hotel Name"]
    city = row["City"]
    country = row["Country"]
    address = row["Hotel Address"]
    #append to the messages list
    messages.append(f"Name: {hotel}, City: {city}, Address: {address}, Country : {country}")

messages

['Name: Best Western Danbury/Bethel, City: bethel, Address: 11 Stony Hill Rd, Bethel, Country : Tulsa County',
 'Name: Chenango Valley State Park, City: broome, Address: 153 State Park Rd, Chenango Forks, Country : Dickinson',
 'Name: St. James Hotel, City: red wing, Address: 406 Main St, Red Wing, Country : Owasso',
 'Name: Resort Uhenha, City: saurimo, Address: Rua da Piscina, bairro Agostinho Neto, Country : AO',
 'Name: HOSTEL A COROA, City: arraial do cabo, Address: R. Marcílio Dias, 217 - Macedônia, Arraial do Cabo, Country : BR',
 'Name: Guest House Ruslan Nurata & Tours, City: nurota, Address: St. Okhunboboyev, h. 2 Nurata UZ, Country : UZ',
 'Name: Best Western Plus Humboldt Bay Inn, City: eureka, Address: 232 W 5th St, Eureka, Country : KS',
 'Name: Kangcheng Jian Guo International Hotel, City: korla, Address: No 618 East Jiaotong Road Korla Xinjiang, Korla, Country : CN',
 'Name: Pine Manor, City: carbondale, Address: 26 Pine Lake Dr, Carbondale, Country : IL',
 'Name: Fresh

In [29]:
#Lets create the layer

#Create position of markers
markers_location = vacation_df[["Latitude", "Longitude"]]

#Create the layer
markers = gmaps.marker_layer(markers_location, info_box_content= [f"{message}" for message in messages])

                                                          
#Lets add the layer to the last map
fig.add_layer(markers)


In [30]:
fig

Figure(layout=FigureLayout(height='420px'))